In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# 데이터 임포트
import os
import pandas as pd

os.chdir("/content/drive/MyDrive/open")

train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')

In [ ]:
# 필요한 모듈 임포트
import pandas as pd
import numpy as np

from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# ID 떼기 (한 번만 실행!)
train_df = train_df.iloc[:, 1:]
val_df = val_df.iloc[:, 1:]

In [ ]:
train_df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,-0.994972
1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,-0.994972
2,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,-0.256131,-0.994960
3,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,0.262698,-0.994901
4,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0.994900,-0.994901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113837,-12.516732,10.187818,-8.476671,-2.510473,-4.586669,-1.394465,-3.632516,5.498583,4.893089,8.655320,...,-0.944759,-1.565026,0.890675,-1.253276,1.786717,0.320763,2.090712,1.232864,-0.169496,1.034857
113838,1.884849,-0.143540,-0.999943,1.506772,-0.035300,-0.613638,0.190241,-0.249058,0.666458,0.120908,...,0.144008,0.634646,-0.042114,-0.053206,0.316403,-0.461441,0.018265,-0.041068,0.530986,1.034881
113839,-0.241923,0.712247,0.399806,-0.463406,0.244531,-1.343668,0.929369,-0.206210,0.106234,-0.284708,...,-0.228876,-0.514376,0.279598,0.371441,-0.559238,0.113144,0.131507,0.081265,-0.230699,1.034904
113840,0.120316,0.931005,-0.546012,-0.745097,1.130314,-0.235973,0.812722,0.115093,-0.204064,-0.657422,...,-0.314205,-0.808520,0.050343,0.102800,-0.435870,0.124079,0.217940,0.068803,-0.269825,1.034939


In [ ]:
# val 데이터에서 사기 거래 비율 추론
val_normal, val_fraud = val_df['Class'].value_counts()
contamination_ratio = val_fraud / val_normal
contamination_ratio

0.0010551491277433877

In [ ]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    #예측된 모든 정상 샘플(격리 포레스트 모델에서 1로 레이블 지정)을 0으로 대체
    model_pred = np.where(model_pred == 1, 0, model_pred)

    #예측된 모든 이상 샘플(격리 포레스트 모델에 의해 -1로 레이블 지정됨)을 1로 대체
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [ ]:
# val 데이터 feature, class 분리
# X_val = val_df.drop(['Class'], axis=1).values
y_val = val_df['Class'].values

In [ ]:
val_df_X = val_df.drop(['Class'], axis=1)
val_df_X

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,-0.255991,-0.994878
1,0.962496,0.328461,-0.171479,2.109204,1.129566,1.696038,0.107712,0.521502,-1.191311,0.724396,...,0.143997,0.402492,-0.048508,-1.371866,0.390814,0.199964,0.016371,-0.014605,0.168937,-0.994784
2,1.145524,0.575068,0.194008,2.598192,-0.092210,-1.044430,0.531588,-0.241888,-0.896287,0.757952,...,0.011106,-0.119703,-0.076510,0.691320,0.633984,0.048741,-0.053192,0.016251,0.169496,-0.994502
3,0.927060,-0.323684,0.387585,0.544474,0.246787,1.650358,-0.427576,0.615371,0.226278,-0.225495,...,-0.040513,0.079359,0.096632,-0.992569,0.085096,0.377447,0.036096,-0.005960,0.331307,-0.994467
4,-3.005237,2.600138,1.483691,-2.418473,0.306326,-0.824575,2.065426,-1.829347,4.009259,6.051521,...,-0.852309,-0.181268,-0.163747,0.515821,0.136318,0.460054,-0.251259,-1.105751,-0.287012,-0.994373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28457,-0.546378,1.433992,-0.313252,0.926044,0.522388,-0.565669,1.066075,0.269799,-1.099446,-0.077753,...,0.228159,0.778817,-0.175451,0.030397,0.018381,-0.395994,0.301655,0.173585,0.202753,1.034622
28458,-0.764523,0.588379,-0.907599,-0.418847,0.901528,-0.760802,0.758545,0.414698,-0.730854,-1.245088,...,0.003530,-0.431876,0.141759,0.587119,-0.200998,0.267337,-0.152951,-0.065285,0.810452,1.034693
28459,-0.446951,1.302212,-0.168583,0.981577,0.578957,-0.605641,1.253430,-1.042610,-0.417116,0.076605,...,0.851800,0.305268,-0.148093,-0.038712,0.010209,-0.362666,0.503092,0.229921,0.537972,1.034763
28460,2.039560,-0.175233,-1.196825,0.234580,-0.008713,-0.726571,0.017050,-0.118228,0.435402,0.267772,...,-0.268048,-0.717211,0.297930,-0.359769,-0.315610,0.201114,-0.080826,-0.075071,-0.269964,1.034928


In [ ]:
# train 데이터셋에 맞춰 훈련
IF = IsolationForest(n_estimators=125, max_samples=len(train_df), contamination=contamination_ratio, 
                        random_state=70, verbose=0)
IF.fit(train_df.to_numpy())

# val 데이터셋 예측
y_pred = IF.predict(val_df_X)
y_pred = get_pred_label(y_pred)

# f1 score 평가
val_score = f1_score(y_val, y_pred, average='macro')

print(f'Validation F1 Score : [{val_score}]')

Validation F1 Score : [0.6829992027762146]


## IF scaling (standard, robust, minmax)

In [ ]:
# Standard Scaler - 정규분포 형태로

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

train_df_scaled = std_scaler.fit_transform(train_df)
val_df_X_scaled = std_scaler.transform(val_df_X)

In [ ]:
# train 데이터셋으로 훈련
IF = IsolationForest(n_estimators=125, max_samples=len(train_df), contamination=contamination_ratio, 
                        random_state=70, verbose=0)
IF.fit(train_df_scaled)

# val 데이터셋 클래스 예측
y_pred = IF.predict(val_df_X)
y_pred = get_pred_label(y_pred)

# f1 score 평가
val_score = f1_score(y_val, y_pred, average='macro')

print(f'Validation F1 Score (Standard Scaler) : [{val_score}]')

Validation F1 Score (Standard Scaler) : [0.6688526408781778]


In [ ]:
# Robust Scaler - 이상치에 덜 민감
from sklearn.preprocessing import RobustScaler

robust_scaler = RobustScaler()

train_df_scaled = robust_scaler.fit_transform(train_df)
val_df_X_scaled = robust_scaler.transform(val_df_X)

In [ ]:
# train 데이터셋으로 훈련
IF = IsolationForest(n_estimators=125, max_samples=len(train_df), contamination=contamination_ratio, 
                        random_state=70, verbose=0)
IF.fit(train_df_scaled)

# val 데이터셋 클래스 예측
y_pred = IF.predict(val_df_X)
y_pred = get_pred_label(y_pred)

# f1 score 평가
val_score = f1_score(y_val, y_pred, average='macro')

print(f'Validation F1 Score (Robust Scaler) : [{val_score}]')

Validation F1 Score (Robust Scaler) : [0.6968049303330581]


In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()

train_df_scaled = minmax_scaler.fit_transform(train_df)
val_df_X_scaled = minmax_scaler.transform(val_df_X)

In [ ]:
# train 데이터셋으로 훈련
IF = IsolationForest(n_estimators=125, max_samples=len(train_df), contamination=contamination_ratio, 
                        random_state=70, verbose=0)
IF.fit(train_df_scaled)

# val 데이터셋 클래스 예측
y_pred = IF.predict(val_df_X)
y_pred = get_pred_label(y_pred)

# f1 score 평가
val_score = f1_score(y_val, y_pred, average='macro')

print(f'Validation F1 Score (MinMax Scaler) : [{val_score}]')

Validation F1 Score (MinMax Scaler) : [0.0010529271374420891]


## IF 파라미터 조정

In [ ]:
# 먼저 robust scaler 사용
from sklearn.preprocessing import RobustScaler

robust_scaler = RobustScaler()

train_df_scaled = robust_scaler.fit_transform(train_df)
val_df_X_scaled = robust_scaler.transform(val_df_X)

In [ ]:
# n_estimators 파라미터 조정
for i in range(100, 1000, 100):
  IF = IsolationForest(n_estimators=i, max_samples=len(train_df_scaled), contamination=contamination_ratio, 
                        random_state=70, verbose=0)
  IF.fit(train_df_scaled)

  y_pred = IF.predict(val_df_X_scaled)
  y_pred = get_pred_label(y_pred)

  val_score = f1_score(y_val, y_pred, average='macro')

  print(f'Validation F1 Score (n_estimator {i}) : [{val_score}]')

Validation F1 Score (n_estimator 100) : [0.6861153460396092]
Validation F1 Score (n_estimator 200) : [0.6861153460396092]
Validation F1 Score (n_estimator 300) : [0.6963959681062595]
Validation F1 Score (n_estimator 400) : [0.699683455261677]
Validation F1 Score (n_estimator 500) : [0.699683455261677]
Validation F1 Score (n_estimator 600) : [0.699683455261677]
Validation F1 Score (n_estimator 700) : [0.7030820840915222]
Validation F1 Score (n_estimator 800) : [0.6963959681062595]
Validation F1 Score (n_estimator 900) : [0.6963959681062595]


In [ ]:
# n_estimator = 700으로 지정
IF = IsolationForest(n_estimators=700, max_samples=len(train_df_scaled), contamination=contamination_ratio, 
                        random_state=70, verbose=0)
IF.fit(train_df_scaled)

y_pred = IF.predict(val_df_X_scaled)
y_pred = get_pred_label(y_pred)

val_score = f1_score(y_val, y_pred, average='macro')

print(f'Validation F1 Score : [{val_score}]')
print(classification_report(y_val, y_pred))

Validation F1 Score : [0.7030820840915222]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.41      0.40      0.41        30

    accuracy                           1.00     28462
   macro avg       0.71      0.70      0.70     28462
weighted avg       1.00      1.00      1.00     28462



## IF feature 조정

전체 feature 사용 -> Validation F1 Score : [0.7030820840915222]

In [ ]:
# PCA 진행
from sklearn.decomposition import PCA

pca = PCA(n_components=25)

train_df_pca = pca.fit_transform(train_df)
train_df_pca = pd.DataFrame(train_df_pca)

val_df_X_pca = pca.transform(val_df_X)
val_df_X_pca = pd.DataFrame(val_df_X_pca)

In [ ]:
# 정규화
from sklearn.preprocessing import RobustScaler

robust_scaler = RobustScaler()

train_df_pca_scaled = robust_scaler.fit_transform(train_df_pca)
val_df_X_pca_scaled = robust_scaler.transform(val_df_X_pca)

In [ ]:
IF.fit(train_df_pca_scaled)

IsolationForest(contamination=0.0010551491277433877, max_samples=113842,
                n_estimators=700, random_state=70)

In [ ]:
# 평가
y_pred_pca = IF.predict(val_df_X_pca_scaled)
y_pred_pca = get_pred_label(y_pred_pca)

val_score_pca = f1_score(y_val, y_pred_pca, average='macro')
print(f'Validation F1 Score (PCA) : [{val_score_pca}]')

Validation F1 Score (PCA) : [0.7102361650869573]


In [ ]:
# feature selecion
selected_feature = [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 16, 17, 18, 21, 27, 30]
feature_num = [selected_feature[i] - 1 for i in range(len(selected_feature))]

train_df_2 = train_df.iloc[:, feature_num]
val_df_X_2 = val_df_X.iloc[:, feature_num]

In [ ]:
# 정규화
from sklearn.preprocessing import RobustScaler

robust_scaler = RobustScaler()

train_df_2_scaled = robust_scaler.fit_transform(train_df_2)
val_df_X_2_scaled = robust_scaler.transform(val_df_X_2)

In [ ]:
IF.fit(train_df_2_scaled)

IsolationForest(contamination=0.0010551491277433877, max_samples=113842,
                n_estimators=700, random_state=70)

In [ ]:
# 평가
y_pred_2 = IF.predict(val_df_X_2_scaled)
y_pred_2 = get_pred_label(y_pred_2)

val_score_2 = f1_score(y_val, y_pred_2, average='macro')
print(f'Validation F1 Score (feature selection) : [{val_score_2}]')

Validation F1 Score (feature selection) : [0.7578007195689396]


In [ ]:
# 스코어 구체적으로
from sklearn.metrics import classification_report

print("모든 feature 사용한 버전")
print(classification_report(y_val, y_pred))
print("===========================")
print("PCA 진행한 버전")
print(classification_report(y_val, y_pred_pca))
print("===========================")
print("feature selection 진행한 버전")
print(classification_report(y_val, y_pred_2))

모든 feature 사용한 버전
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.41      0.40      0.41        30

    accuracy                           1.00     28462
   macro avg       0.71      0.70      0.70     28462
weighted avg       1.00      1.00      1.00     28462

PCA 진행한 버전
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.44      0.40      0.42        30

    accuracy                           1.00     28462
   macro avg       0.72      0.70      0.71     28462
weighted avg       1.00      1.00      1.00     28462

feature selection 진행한 버전
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.50      0.53      0.52        30

    accuracy                           1.00     28462
   macro avg       0.75      0.77      0.76     28462
weighted avg       1